[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [14]:
from ecdsa import SigningKey, SECP256k1

privateKey = SigningKey.generate(SECP256k1)
publicKey = privateKey.get_verifying_key()

message = b"bitcoin is life"
signature = privateKey.sign(message)

publicKey.verify(signature, message)

True

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [3]:
class Transfer:
    def __init__(self, signature, publicKey):
        self.signature = signature
        self.publicKey = publicKey
        
class ECDSACoin:
    def __init__(self, transfers):
        self.transfers = transfers

In [11]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [22]:
from utils import serialize

# given a public key, return an ECDSACoin
# bank --ECDSACoin--> publicKey provided
def issue(publicKey):
    message = serialize(publicKey)
    signature = bank_private_key.sign(message)
    transfer = Transfer(
        signature=signature,
        publicKey=publicKey
    )
    coin = ECDSACoin([transfer])
    return coin

# Validating the First Transfer

In [17]:
def validate_first_transfer(coin):
    transfer = coin.transfers[0]
    bank_public_key.verify(transfer.signature, serialize(transfer.publicKey))

In [28]:
aliceCoin = issue(alice_public_key)

In [29]:
validate_first_transfer(aliceCoin)

True

In [30]:
from ecdsa import BadSignatureError

message = serialize(bob_public_key)
signature = bob_private_key.sign(message)
transfer = Transfer(
    signature=signature,
    publicKey=bob_public_key
)
bobCoin = ECDSACoin([transfer])
try:
    validate_first_transfer(bobCoin)
except BadSignatureError:
    print("Bad signature detected")

Bad signature detected


# Validating Subsequent Transfers

In [31]:
def transfer_message(previous_signature, next_owner_public_key):
    return serialize({
        "previous_signature": previous_signature,
        "next_owner_public_key": next_owner_public_key
    })

def validate(coin):
    #check the first tranfer
    transfer = coin.transfers[0]
    bank_public_key.verify(transfer.signature, serialize(transfer.publicKey))
    
    prev_transfer = transfer
    for next_transfer in coin.transfers[1:]:
        prev_transfer.publicKey.verify(
            next_transfer.signature, 
            transfer_message(prev_transfer.signature,next_transfer.publicKey)
        )
        prev_transfer = next_transfer

In [32]:
def get_owner(coin):
    database = {
        bank_public_key: "Bank",
        bob_public_key: "Bob",
        alice_public_key: "Alice"
    }
    return database[coin.transfers[-1].publicKey]

In [41]:
coin = issue(alice_public_key)
validate(coin)
print(get_owner(coin))
message = transfer_message(coin.transfers[-1].signature,bob_public_key)
alice_to_bob = Transfer(
    signature=alice_private_key.sign(message),
    publicKey=bob_public_key
)
coin.transfers.append(alice_to_bob)
validate(coin)
print(get_owner(coin))

Alice
Bob


# Serialization

In [42]:
from utils import to_disk, from_disk

In [44]:
import os

filename = "coin.ecdsacoin"

print("does coinfile exist on disk?", os.path.isfile(filename))

does coinfile exist on disk? False


In [45]:
coin = issue(alice_public_key)
to_disk(coin, filename)

In [46]:
print("does coinfile exist on disk?", os.path.isfile(filename))

does coinfile exist on disk? True


In [48]:
coin= from_disk(filename)
coin

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(coin.transfers[0].signature),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()